In [1]:
import pandas as pd
path = r'..\Datasets\Allergen_Status_Encoded.csv'
data=pd.read_csv(path)
data

,Price ($),Customer rating (Out of 5),Prediction,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens
0,10.15,3.1,1.0,1.000000,1.000000,0.890110,0.940476,1.000000,1.000000
1,6.17,4.5,1.0,1.000000,1.000000,0.890110,0.940476,1.000000,1.000000
2,19.65,4.1,1.0,0.333333,1.000000,0.543165,0.631579,0.900000,1.000000
3,17.48,4.7,1.0,0.333333,1.000000,0.543165,0.631579,0.900000,1.000000
4,10.83,3.7,1.0,0.640704,1.000000,0.543165,0.631579,0.900000,1.000000
...,...,...,...,...,...,...,...,...,...
393,5.07,2.9,1.0,1.000000,0.500000,0.890110,0.940476,1.000000,1.000000
394,11.95,4.4,1.0,0.640704,0.640704,0.890110,0.940476,0.640704,1.000000
395,12.67,3.4,1.0,1.000000,0.666667,0.890110,0.940476,1.000000,1.000000
396,15.83,2.4,1.0,1.000000,1.000000,0.890110,0.940476,1.000000,1.000000


In [2]:
from sklearn.model_selection import train_test_split

data_cleaned = data

X = data_cleaned.drop(columns='Prediction')
y = data_cleaned['Prediction']
print("Shape of X (Features):", X.shape)
print("Shape of y (Target):", y.shape)
X_train_80, X_test_80, y_train_80, y_test_80 = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_70, X_test_70, y_train_70, y_test_70 = train_test_split(X, y, test_size=0.3, random_state=42)

print("\n80:20 Split Shapes:")
print("X_train:", X_train_80.shape)
print("X_test:", X_test_80.shape)
print("y_train:", y_train_80.shape)
print("y_test:", y_test_80.shape)

print("\n70:30 Split Shapes:")
print("X_train:", X_train_70.shape)
print("X_test:", X_test_70.shape)
print("y_train:", y_train_70.shape)
print("y_test:", y_test_70.shape)


Shape of X (Features): (398, 8)
Shape of y (Target): (398,)

80:20 Split Shapes:
X_train: (318, 8)
X_test: (80, 8)
y_train: (318,)
y_test: (80,)

70:30 Split Shapes:
X_train: (278, 8)
X_test: (120, 8)
y_train: (278,)
y_test: (120,)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import pandas as pd

models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Gaussian Naive Bayes": GaussianNB()
}

results = {
    "Model": [],
    "Split": [],
    "Training Accuracy": [],
    "Testing Accuracy": []
}

def train_and_evaluate(model, X_train, X_test, y_train, y_test, split_name):
    model.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, model.predict(X_test))
    return train_acc, test_acc

for model_name, model in models.items():
    train_acc, test_acc = train_and_evaluate(model, X_train_80, X_test_80, y_train_80, y_test_80, "80:20")
    results["Model"].append(model_name)
    results["Split"].append("80:20")
    results["Training Accuracy"].append(train_acc)
    results["Testing Accuracy"].append(test_acc)

for model_name, model in models.items():
    train_acc, test_acc = train_and_evaluate(model, X_train_70, X_test_70, y_train_70, y_test_70, "70:30")
    results["Model"].append(model_name)
    results["Split"].append("70:30")
    results["Training Accuracy"].append(train_acc)
    results["Testing Accuracy"].append(test_acc)

results_df = pd.DataFrame(results)

print(results_df)


                  Model  Split  Training Accuracy  Testing Accuracy
0         Random Forest  80:20           1.000000             1.000
1         Decision Tree  80:20           1.000000             1.000
2   K-Nearest Neighbors  80:20           0.981132             0.950
3  Gaussian Naive Bayes  80:20           1.000000             1.000
4         Random Forest  70:30           1.000000             1.000
5         Decision Tree  70:30           1.000000             1.000
6   K-Nearest Neighbors  70:30           0.978417             0.975
7  Gaussian Naive Bayes  70:30           1.000000             1.000


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_80)
X_test = scaler.transform(X_test_80)

param_grid = {
    'n_neighbors': np.arange(1, 30), 
    'weights': ['uniform', 'distance'], 
    'p': [1, 2]
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1, 
    n_jobs=-1 
)


grid_search.fit(X_train_80, y_train_80)

best_knn = grid_search.best_estimator_

train_predictions = best_knn.predict(X_train_80)
train_accuracy = accuracy_score(y_train_80, train_predictions)

test_predictions = best_knn.predict(X_test_80)
test_accuracy = accuracy_score(y_test_80, test_predictions)

print("Best Parameters:", grid_search.best_params_)
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)




Fitting 5 folds for each of 116 candidates, totalling 580 fits
Best Parameters: {'n_neighbors': np.int64(9), 'p': 1, 'weights': 'uniform'}
Training Accuracy: 0.9874213836477987
Testing Accuracy: 0.975


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [5]:
import joblib
joblib.dump(best_knn, r'..\Models\best_knn_model.pkl')

['..\\Models\\best_knn_model.pkl']